TODO:
- second example and bias axis (next to gender)
- hypothesis
- BERT
- Repo-Link

# Embeddings

In [1]:
from utils import *
from scipy.spatial.distance import cosine

2022-03-18 18:51:28.851034: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-18 18:51:28.851128: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## GloVe (Global Vectors for Word Representation)

Paper: https://nlp.stanford.edu/pubs/glove.pdf

Pre-trained word vectors: https://nlp.stanford.edu/projects/glove/

In [2]:
# Read GloVe vectors into a word-to-vec dictionary
glove_map = read_glove_vecs("data/glove.6B.50d.txt")

400000it [00:06, 60743.08it/s]


## BERT (Bidirectional Encoder Representations from Transformers)
Paper: https://arxiv.org/pdf/1810.04805.pdf

Implementation: https://mccormickml.com/2019/05/14/BERT-word-embeddings-tutorial/

In [21]:
vocab = ["man", "doctor", "woman", "nurse", "italy", "italian", "spain", "india", "delhi", "japan", "man", "woman", "boy", "small", "smaller", "large", "mother", "father", "girl", "boy", "john", "marie", "sophie", "ronaldo", "priya", "rahul", "danielle", "reza", "katy", "yasmin", "lipstick", "guns", "science", "arts", "literature", "warrior","doctor", "tree", "receptionist", "technology",  "fashion", "teacher", "engineer", "pilot", "computer", "singer", "receptionist"]
bert_map = create_bert_map(vocab)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 47/47 [00:06<00:00,  7.14it/s]


## Cosine similarity and cosine distance

The cosine similarity is a measure for similarity between two vectors. It always belongs to the interval [-1, 1] and is defined as follows:

$$\text{CosineSimilarity(u, v)} = \frac {u \cdot v} {||u||_2 ||v||_2} = cos(\theta) \tag{1}$$


The cosine distance lies in [0, 2] and is defined by 

$$\text{CosineDistance(u, v)} = 1 - \text{CosineSimilarity(u, v)}\tag{2}$$


In [3]:
dyads = [
    ("man", "doctor"),
    ("woman", "nurse"),
    ("man", "woman"),
    ("doctor", "nurse"),
    ("man", "nurse"),
    ("man", "nurse"),
    ("woman", "doctor")
]

quartets = [
    ("man", "doctor", "woman", "nurse"),
    ("man", "nurse", "woman", "doctor")
]

# Print cosine similarity for each dyad
for dyad in dyads:
    print("{:<13}{:>6,.2f}".format(str(dyad[0]) + "-" + str(dyad[1]) + ":", cosine(glove_map[dyad[0]], glove_map[dyad[1]])))

print()

# Print cosine similarity between the dyads in each quartet to demonstrate gender bias
for quartet in quartets:
    print("{} -> {} :: {} -> {}: {:>6,.2f}".format(*quartet, cosine(glove_map[quartet[0]], glove_map[quartet[1]]) - cosine(glove_map[quartet[2]], glove_map[quartet[3]])))

man-doctor:    0.29
woman-nurse:   0.28
man-woman:     0.11
doctor-nurse:  0.20
man-nurse:     0.43
man-nurse:     0.43
woman-doctor:  0.27

man -> doctor :: woman -> nurse:   0.00
man -> nurse :: woman -> doctor:   0.15


In [4]:
# Define words for which embeddings should be visualized
words = ["man", "doctor", "woman", "nurse"]

# Create word-to-vec map
map = {}
for word in words:
    map[word] = glove_map[word]

# Create the log file for the TensorBoard visualization
visualize_embedding(word_to_vec_map=map, dir_name="glove_gender")

2022-03-18 18:51:39.968904: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-18 18:51:39.968996: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-18 18:51:39.969068: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (zenbook): /proc/driver/nvidia/version does not exist
2022-03-18 18:51:39.969702: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
%tensorboard # select logs dir and go to "Projector"

UsageError: Line magic function `%tensorboard` not found.


## Analogies

Performs the word analogy task "a is to b as c is to ...".

In [8]:
# Define triads to try analogies
triads = [
    ("italy", "italian", "spain"),
    ("india", "delhi", "japan"),
    ("man", "woman", "boy"),
    ("small", "smaller", "large")
]

# Print results
for triad in triads:
    print ("{} -> {} :: {} -> {}".format(*triad, complete_analogy(*triad, glove_map)))

italy -> italian :: spain -> spanish
india -> delhi :: japan -> tokyo
man -> woman :: boy -> girl
small -> smaller :: large -> smaller


In [9]:
# Define words for which embeddings should be visualized
words = ["italy", "italian", "spain", "spanish", "india", "delhi", "japan", "tokyo"]

# Create word-to-vec map
map = {}
for word in words:
    map[word] = glove_map[word]

# Create the log file for the TensorBoard visualization
visualize_embedding(word_to_vec_map=map, dir_name="glove_countries")

In [ ]:
%tensorboard # select logs dir and go to "Projector"

# Debiasing

The debiasing algorithm is from Bolukbasi et al. (2016), Man is to Computer Programmer as Woman is to
Homemaker? Debiasing Word Embeddings (https://arxiv.org/abs/1607.06520)

#### 1. Identify bias direction (e.g. gender)
- e(he) - e(she)
- e(male) - e(female)
- ...
- Average = bias direction of gender

In [10]:
# Calculate distance between gender pair woman-man
woman_man = glove_map["woman"] - glove_map["man"]

# Calculate distance between gender pair mother-father
mother_father = glove_map["mother"] - glove_map["father"]

# Calculate distance between gender pair girl-boy
girl_boy = glove_map["girl"] - glove_map["boy"]

# Average over the gender pairs to get a simple representation of gender
gender = np.average([woman_man, mother_father, girl_boy], axis=0)

In [11]:
# Define girls and boys names for comparing the gender similarity
name_list = ["john", "marie", "sophie", "ronaldo", "priya", "rahul", "danielle", "reza", "katy", "yasmin"]

# Print results
for w in name_list:
    print("{:<9}{:>6,.2f}".format(w + ":", cosine(glove_map[w], gender)))

john:      1.31
marie:     0.66
sophie:    0.59
ronaldo:   1.29
priya:     0.80
rahul:     1.19
danielle:  0.71
reza:      1.17
katy:      0.69
yasmin:    0.80


In [12]:
# Define random words for comparing the gender similarity
word_list = ["lipstick", "guns", "science", "arts", "literature", "warrior","doctor", "tree", "receptionist", 
             "technology",  "fashion", "teacher", "engineer", "pilot", "computer", "singer"]

# Print results (and look at the gender stereotypes!)
for w in word_list:
    print("{:<13}{:>6,.2f}".format(w + ":", cosine(glove_map[w], gender)))

lipstick:      0.59
guns:          1.09
science:       1.06
arts:          0.99
literature:    0.98
warrior:       1.17
doctor:        0.92
tree:          0.96
receptionist:  0.70
technology:    1.16
fashion:       0.86
teacher:       0.89
engineer:      1.23
pilot:         1.04
computer:      1.17
singer:        0.80


#### 2. Neutralize gender-neutral words
- gender-intrinsic (e.g. girl/boy, he/she) vs. gender-neutral (e.g. doctor, babysitter)
- linear classifier to identify which words should be neutralized

$$e^{bias\_component} = \frac{e \cdot g}{||g||_2^2} * g\tag{3}$$
$$e^{debiased} = e - e^{bias\_component}\tag{4}$$

In [20]:
print("Cosine similarity before equalizing:")
print("{:<20}{:>6,.2f}".format("receptionist-gender:", cosine(glove_map["receptionist"], gender)))

e_debiased = neutralize("receptionist", gender, glove_map)

print()
print("Cosine similarity after equalizing:")
print("{:<13}{:>6,.2f}".format("receptionist-gender:", cosine(e_debiased, gender)))

Cosine similarity before equalizing:
receptionist-gender:  0.70

Cosine similarity after equalizing:
receptionist-gender:  1.00


#### 3. Equalize pairs
- e.g. grandmother and grandfather should have the same distance from gender-neutral words
- hand-pick pairs to be equalized

$$ \mu = \frac{e_{w1} + e_{w2}}{2} \tag{5} $$ 

$$ \mu_{B} = \frac {\mu \cdot \text{bias axis}}{||\text{bias axis}||_2^2} *\text{bias axis} \tag{6} $$

$$ \mu_{\perp} = \mu - \mu_{B} \tag{7} $$

$$ e_{w1B} = \frac {e_{w1} \cdot \text{bias axis}}{||\text{bias axis}||_2^2} *\text{bias axis} \tag{8} $$

$$ e_{w2B} = \frac {e_{w2} \cdot \text{bias axis}}{||\text{bias axis}||_2^2} *\text{bias axis} \tag{9} $$

$$e_{w1B}^{corrected} = \sqrt{ |{1 - ||\mu_{\perp} ||^2_2} |} * \frac{e_{\text{w1B}} - \mu_B} {||(e_{w1} - \mu_{\perp}) - \mu_B||_2} \tag{10} $$

$$e_{w2B}^{corrected} = \sqrt{ |{1 - ||\mu_{\perp} ||^2_2} |} * \frac{e_{\text{w2B}} - \mu_B} {||(e_{w2} - \mu_{\perp}) - \mu_B||_2} \tag{11} $$

$$e_1 = e_{w1B}^{corrected} + \mu_{\perp} \tag{12} $$

$$e_2 = e_{w2B}^{corrected} + \mu_{\perp} \tag{13} $$

In [18]:
print("Cosine similarities before equalizing:")
print("{:<13}{:>6,.2f}".format("man-gender:", cosine(glove_map["man"], gender)))
print("{:<13}{:>6,.2f}".format("woman-gender:", cosine(glove_map["woman"], gender)))

e1, e2 = equalize(("man", "woman"), gender, glove_map)

print()
print("Cosine similarities after equalizing:")
print("{:<13}{:>6,.2f}".format("man-gender:", cosine(e1, gender)))
print("{:<13}{:>6,.2f}".format("woman-gender:", cosine(e2, gender)))

Cosine similarities before equalizing:
man-gender:    1.02
woman-gender:  0.60

Cosine similarities after equalizing:
man-gender:    1.66
woman-gender:  0.34
